In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [6]:
# ==========================
#  Load train.npz
# ==========================
data = np.load("train.npz")
x = data["x"]     # shape: (39163, 28, 28) 또는 (39163, 28, 28, C)
y = data["y"]     # 문자열 라벨

print("Raw x shape:", x.shape)
print("Raw y shape:", y.shape)
print("Unique classes:", len(np.unique(y)))


Raw x shape: (39163, 28, 28)
Raw y shape: (39163,)
Unique classes: 5


In [7]:
le = LabelEncoder()
y_idx = le.fit_transform(y)
num_classes = len(le.classes_)

print("num_classes:", num_classes)


num_classes: 5


In [8]:
# float32로 변환
x = x.astype("float32")

# grayscale → RGB 변환
if x.ndim == 3:               # (N, H, W)
    x = x[..., np.newaxis]

if x.shape[-1] == 1:          # (N, H, W, 1)
    x = np.repeat(x, 3, axis=-1)

print("Converted shape:", x.shape)


Converted shape: (39163, 28, 28, 3)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(
    x, y_idx,
    test_size=0.2,
    random_state=42,
    stratify=y_idx
)

print("train:", x_train.shape)
print("valid:", x_val.shape)


train: (31330, 28, 28, 3)
valid: (7833, 28, 28, 3)


In [10]:
IMG_SIZE = 128      # 메모리 아껴도 정확도 잘 나오는 해상도
batch_size = 32

# train 전처리 함수
def preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    return img, label

# val 전처리 함수
def preprocess_val(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    return img, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(5000)
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((x_val, y_val))
    .map(preprocess_val, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)


In [11]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# augmentation (가벼운 버전)
aug = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
])

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x_in = aug(inputs)

# EfficientNet 전처리
x_in = preprocess_input(x_in)

# EfficientNetB0 backbone
base = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_tensor=x_in
)
base.trainable = False   # 1단계: 상단만 학습

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 128, 128,  │          0 │ sequential[0][0]  │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 128, 128,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 128, 128,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 129, 129,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 64, 64,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 64, 64,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 64, 64,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 64, 64,    │          0 │ block1a_activati

 Total params: 4,055,976 (15.47 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

es = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[es, rlr],
    verbose=1
)

val_loss, val_acc = model.evaluate(val_ds)
print("Validation acc:", val_acc)


Epoch 1/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 1006s 1s/step - accuracy: 0.2057 - loss: 1.6348 - val_accuracy: 0.1829 - val_loss: 1.6282 - learning_rate: 0.0010
Epoch 2/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 968s 988ms/step - accuracy: 0.2117 - loss: 1.6265 - val_accuracy: 0.2548 - val_loss: 1.5982 - learning_rate: 0.0010
Epoch 3/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 992s 1s/step - accuracy: 0.2177 - loss: 1.6204 - val_accuracy: 0.2043 - val_loss: 1.6116 - learning_rate: 0.0010
Epoch 4/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 1004s 1s/step - accuracy: 0.2231 - loss: 1.6202 - val_accuracy: 0.3816 - val_loss: 1.5973 - learning_rate: 0.0010
Epoch 5/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 995s 1s/step - accuracy: 0.2258 - loss: 1.6140 - val_accuracy: 0.2043 - val_loss: 1.6122 - learning_rate: 0.0010
Epoch 6/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 1016s 1s/step - accuracy: 0.2243 - loss: 1.6153 - val_accuracy: 0.2043 - val_loss: 1.6093 - learning_rate: 0.0010
Epoch 7/20
980/980 ━━━━━━━━━━━━━━━━━━━━ 1017s 1s/step - accuracy: 0.2236 - l

In [ ]:
base.trainable = True
fine_start = len(base.layers) - 30   # 마지막 30개 레이어만 학습

for i, layer in enumerate(base.layers):
    layer.trainable = (i >= fine_start)

model.compile(
    optimizer=Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[es],
    verbose=1
)

val_loss, val_acc = model.evaluate(val_ds)
print("Fine-tuned acc:", val_acc)


In [ ]:
test_data = np.load("test.npz")
x_test = test_data["x"].astype("float32")

# grayscale → RGB
if x_test.ndim == 3:
    x_test = x_test[..., np.newaxis]
if x_test.shape[-1] == 1:
    x_test = np.repeat(x_test, 3, axis=-1)

# tf.data로 처리
test_ds = (
    tf.data.Dataset.from_tensor_slices(x_test)
    .map(lambda img: tf.image.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0,
         num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:
# Predict
probs = model.predict(test_ds)
pred_idx = probs.argmax(axis=1)

# 숫자 → 문자열 라벨
pred_labels = le.inverse_transform(pred_idx)

# Save submission
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)     # 불필요한 Unnamed 컬럼 제거
df["result"] = pred_labels

df.to_csv("new_submission.csv", index=False)
print("✅ new_submission.csv saved!")


#다시

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# 경고 메시지 무시 설정 (선택 사항)
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 로드
print("1. 데이터 로드 중...")
try:
    train_data = np.load("train.npz")
    x_train_full = train_data['x']
    y_train_full = train_data['y']   # 현재는 문자열 라벨 (예: 'Chest', 'Abdomen', ...)

    test_data = np.load("test.npz")
    x_test = test_data['x']

    df_submission = pd.read_csv("submission.csv")
    print("데이터 로드 완료.")
except FileNotFoundError as e:
    print(f"오류: {e}. 필요한 파일이 현재 디렉토리에 있는지 확인하세요.")
    exit()

# y_train_full 정보 확인
print("y_train_full 예시:", y_train_full[:5])

# 문자열 라벨 → 정수 라벨로 변환
label_encoder = LabelEncoder()
y_train_full_encoded = label_encoder.fit_transform(y_train_full)  # 예: ['Chest','Leg',...] → [0,1,...]

num_classes = len(label_encoder.classes_)
image_shape = x_train_full.shape[1:]

print(f"총 클래스(분류할 신체 부위) 개수: {num_classes}")
print("클래스 이름들:", label_encoder.classes_)
print(f"원본 이미지 해상도: {image_shape}")
print(f"전체 학습 샘플 수: {x_train_full.shape[0]}, 시험 샘플 수: {x_test.shape[0]}")

# 2. 데이터 전처리
print("\n2. 데이터 전처리 중...")

# A. 정규화 및 CNN 입력 형태 조정
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 흑백 이미지일 경우, (N, H, W) -> (N, H, W, 1)
if len(image_shape) == 2:
    x_train_full = np.expand_dims(x_train_full, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    input_shape = x_train_full.shape[1:]
    print(f"CNN 입력 shape로 변경 완료: {input_shape}")
else:
    input_shape = x_train_full.shape[1:]

# B. 레이블 원-핫 인코딩 (정수 라벨 사용!)
y_train_full_one_hot = to_categorical(y_train_full_encoded, num_classes=num_classes)

# C. 학습/검증 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_train_full_one_hot,
    test_size=0.1,
    random_state=42
)
print(f"학습 데이터 분할 완료. 학습 샘플: {x_train.shape[0]}, 검증 샘플: {x_val.shape[0]}")


# 3. 합성곱 신경망 (CNN) 모델 설계
print("\n3. CNN 모델 설계 중...")

def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        # 첫 번째 합성곱 블록
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        # 두 번째 합성곱 블록
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        # 세 번째 합성곱 블록
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        # 분류기
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = create_cnn_model(input_shape, num_classes)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4. 모델 학습
print("\n4. 모델 학습 시작 (최적의 정확도 달성을 위해 시간이 소요될 수 있습니다)...")

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f"\n최적 모델의 검증 정확도: {val_acc:.5f}")
if val_acc > 0.90:
    print("Baseline보다 높은 정확도 달성 가능성이 높습니다! (2.0점 목표)")
else:
    print("성능 개선을 위해 모델 구조 또는 하이퍼파라미터를 조정해 보세요.")


# 5. 예측 및 제출 파일 생성
print("\n5. 시험 데이터 예측 및 제출 파일 생성 중...")

# test.npz 데이터에 대한 예측
y_pred_one_hot = model.predict(x_test)
y_pred_int = np.argmax(y_pred_one_hot, axis=1)  # 0 ~ num_classes-1

# 정수 라벨 → 원래 문자열 라벨로 변환
y_pred_labels = label_encoder.inverse_transform(y_pred_int)

# submission.csv 양식에 예측 결과 기록
# 👉 과제에서 'result'에 무엇을 요구하는지에 따라 아래 둘 중 택1
# 1) 문자열 라벨 제출 (예: 'Chest')
df_submission["result"] = y_pred_labels

# 2) 만약 정수 라벨(0~4)을 요구한다면 대신 아래 줄 사용
# df_submission["result"] = y_pred_int

submission_file_name = "my_submission_xai_05.csv"
df_submission.to_csv(submission_file_name, index=False)

print(f"\n✅ 과제 5 완료: 제출 파일 '{submission_file_name}'이(가) 성공적으로 생성되었습니다.")
print("이 파일을 스마트클래스에 제출하시면 됩니다.")


1. 데이터 로드 중...
데이터 로드 완료.
y_train_full 예시: ['Chest' 'Breast' 'Hand' 'Breast' 'Abdomen']
총 클래스(분류할 신체 부위) 개수: 5
클래스 이름들: ['Abdomen' 'Breast' 'Chest' 'Hand' 'Head']
원본 이미지 해상도: (28, 28)
전체 학습 샘플 수: 39163, 시험 샘플 수: 9791

2. 데이터 전처리 중...
CNN 입력 shape로 변경 완료: (28, 28, 1)
학습 데이터 분할 완료. 학습 샘플: 35246, 검증 샘플: 3917

3. CNN 모델 설계 중...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161,285 (630.02 KB)

 Trainable params: 161,285 (630.02 KB)

 Non-trainable params: 0 (0.00 B)


4. 모델 학습 시작 (최적의 정확도 달성을 위해 시간이 소요될 수 있습니다)...
Epoch 1/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8166 - loss: 0.4393 - val_accuracy: 0.9946 - val_loss: 0.0234
Epoch 2/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9932 - loss: 0.0228 - val_accuracy: 0.9977 - val_loss: 0.0081
Epoch 3/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9965 - loss: 0.0129 - val_accuracy: 0.9992 - val_loss: 0.0045
Epoch 4/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9972 - loss: 0.0075 - val_accuracy: 0.9980 - val_loss: 0.0059
Epoch 5/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9982 - loss: 0.0057 - val_accuracy: 0.9990 - val_loss: 0.0028
Epoch 6/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9981 - loss: 0.0058 - val_accuracy: 0.9982 - val_loss: 0.0044
Epoch 7/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9987 - loss: 0.0044 - val_accuracy: 0.9987 - val_loss: 0.0045
Epoch 8/100
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms

In [5]:
# =============================
# 5. 예측 및 제출 파일 생성
# =============================
print("\n5. 시험 데이터 예측 및 제출 파일 생성 중...")

# test.npz 데이터에 대한 예측
y_pred_one_hot = model.predict(x_test)

# 예측 → 정수 라벨
y_pred_int = np.argmax(y_pred_one_hot, axis=1)

# 정수 라벨 → 문자열 라벨로 변환
y_pred_labels = label_encoder.inverse_transform(y_pred_int)

# submission.csv 양식에 맞게 result 컬럼 채우기
df_submission["result"] = y_pred_labels   # 또는 y_pred_int (과제 요구 형식에 따라 선택)

# 파일 이름 지정 후 저장
submission_file_name = "my_submission_xai_05.csv"
df_submission.to_csv(submission_file_name, index=False, encoding='utf-8')

print(f"\n✅ 제출 파일 저장 완료: {submission_file_name}")



5. 시험 데이터 예측 및 제출 파일 생성 중...
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

✅ 제출 파일 저장 완료: my_submission_xai_05.csv


In [6]:
from google.colab import files
files.download("my_submission_xai_05.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>